In [ ]:
from manim import *

config.media_width = "75%"
config.verbosity = "WARNING"
%%manim -qm ProtonNeutronSeparation


In [2]:
%%manim -qm CircleToSquare

class CircleToSquare(Scene):
    def construct(self):
        blue_circle = Circle(color=BLUE, fill_opacity=0.5)
        green_square = Square(color=GREEN, fill_opacity=0.8)
        self.play(Create(blue_circle))
        self.wait()
        
        self.play(Transform(blue_circle, green_square))
        self.wait()

Manim Community v0.18.1

In [3]:
%%manim -qm ProtonNeutronSeparation

from manim import *
import numpy as np

class ProtonNeutronSeparation(Scene):
    def construct(self):
        # Define angles
        electron_angle = 41.9 * DEGREES
        hadron_angle = -24.7 * DEGREES

        target = Circle(radius=0.3, color=BLUE).move_to(ORIGIN)
        target_label = Text("LD2 Target", font_size=24).next_to(target, DOWN)

        beam_line = Line([-4, 0, 0], [0, 0, 0], color=YELLOW)
        beam_label = Text("e⁻ beam", font_size=24).next_to(beam_line, UP)

        # BigBite setup
        bb_distance = 3
        bb_magnet_pos = [bb_distance * 0.5 * np.cos(electron_angle), bb_distance * 0.5 * np.sin(electron_angle), 0]
        bb_pos = [bb_distance * 0.6 * np.cos(electron_angle), bb_distance * 0.6 * np.sin(electron_angle), 0]
        
        bigbite = Rectangle(height=1, width=0.3, color=GREEN).move_to(bb_pos)
        bb_magnet = Rectangle(height=0.8, width=0.4, color=RED).move_to(bb_magnet_pos)
        bb_label = Text("BigBite", font_size=24).next_to(bb_magnet, RIGHT)

        # Create HCal (higher position)
        hcal_distance = 4
        hcal_shift = 0.6  # Keep HCal shifted up
        hcal_height = 3  # Store height for reference
        
        # Center position of HCal
        hcal_center = [hcal_distance * np.cos(hadron_angle), 
                      hcal_distance * np.sin(hadron_angle) + hcal_shift, 
                      0]
        
        # Calculate hit point (3/4 down from top of HCal)
        hit_point = [
            hcal_center[0],
            hcal_center[1] - hcal_height * 0.2,  # Adjust this factor to move hit point
            0
        ]
        
        # Create HCal rectangle centered at hcal_center
        hcal = Rectangle(height=hcal_height, width=0.3, color=GREEN).move_to(hcal_center)
        hcal_label = Text("HCal", font_size=24).next_to(hcal, DOWN)
        hcal_pos = [hcal_distance * np.cos(hadron_angle), 
                   hcal_distance * np.sin(hadron_angle) + hcal_shift, 
                   0]
        hcal = Rectangle(height=2, width=1, color=GREEN).move_to(hcal_pos)
        hcal_label = Text("HCal", font_size=24).next_to(hcal, DOWN)

        # Create SBS magnet
        sbs_distance = 2
        sbs_pos = [sbs_distance * np.cos(hadron_angle), sbs_distance * np.sin(hadron_angle), 0]
        sbs = Rectangle(height=1, width=0.5, color=RED).move_to(sbs_pos)
        sbs_label = Text("SBS", font_size=24).next_to(sbs, DOWN, buff=0.2)

        # Initial setup
        self.play(
            Create(beam_line),
            Write(beam_label),
            Create(target),
            Write(target_label),
            Create(bigbite),
            Create(bb_magnet),
            Write(bb_label),
            Create(sbs),
            Write(sbs_label),
            Create(hcal),
            Write(hcal_label)
        )
        self.wait(1)

        # Electron path
        electron = Dot(color=YELLOW).move_to(ORIGIN)
        e_path = CubicBezier(
            start_anchor=ORIGIN,
            start_handle=[bb_magnet_pos[0] * 0.5, bb_magnet_pos[1] * 0.5, 0],
            end_handle=[bb_pos[0] + 0.1, bb_pos[1] + 0.1, 0],
            end_anchor=bb_pos,
            color=YELLOW
        )
        
        self.play(
            MoveAlongPath(electron, e_path),
            Create(e_path.copy().set_stroke(opacity=0.5)),
            run_time=2
        )
        self.wait(1)

        # Expected hadron path with dotted line (now to hit_point)
        expected_path = DashedLine(
            start=ORIGIN,
            end=hit_point,
            color=WHITE,
            dash_length=0.1
        )
        
        self.play(Create(expected_path))
        self.wait(10)
        self.play(FadeOut(expected_path))

        # Neutron following expected path
        neutron = Dot(color=BLUE).move_to(ORIGIN)
        n_path = Line(ORIGIN, hit_point, color=BLUE)
        
        self.play(
            MoveAlongPath(neutron, n_path),
            Create(n_path.copy().set_stroke(opacity=0.5)),
            run_time=2
        )
        neutron_label = Text("Neutron", font_size=20, color=BLUE).next_to(neutron, RIGHT, buff=0.5)
        self.play(FadeIn(neutron_label))
        self.wait(5)

        # self.play(
        #     FadeOut(neutron),
        #     FadeOut(neutron_label),
        #     FadeOut(n_path)
        # )

        # Proton path
        proton = Dot(color=RED).move_to(ORIGIN)
        # Proton path - update to use hit_point for initial trajectory
         # Proton path - update to use hit_point for initial trajectory
        def get_point(t):
            # Get point on original straight path to new hit point
            straight_x = t * hit_point[0]
            straight_y = t * hit_point[1]

            # Rest of deflection logic remains the same
            deflection_start = 0.35
            deflection_end = 0.55
            max_deflection = 0.12

            if t <= deflection_start:
                return np.array([straight_x, straight_y, 0])
            elif t <= deflection_end:
                progress = (t - deflection_start) / (deflection_end - deflection_start)
                deflection = max_deflection * (progress * progress)
                return np.array([straight_x, straight_y + deflection, 0])
            else:
                end_deflection = max_deflection
                direction = np.array([1, end_deflection * 0.5, 0])
                direction = direction / np.linalg.norm(direction)
                
                curve_end = np.array([
                    deflection_end * hit_point[0],
                    deflection_end * hit_point[1] + end_deflection,
                    0
                ])
                
                progress = (t - deflection_end) / (1 - deflection_end)
                return curve_end + direction * progress * (hit_point[0] - curve_end[0])

        points = [get_point(t) for t in np.linspace(0, 1, 100)]
        proton_path = VMobject(color=RED)
        proton_path.set_points_smoothly(points)
        
        self.play(
            MoveAlongPath(proton, proton_path),
            Create(proton_path.copy().set_stroke(opacity=0.5)),
            run_time=2
        )
        
        proton_label = Text("Proton", font_size=20, color=RED).next_to(proton, RIGHT, buff=0.5)
        self.play(FadeIn(proton_label))
        self.wait(5)
        
        # Delta position indicator
        delta_line = DashedLine(
            start=hit_point,
            end=proton.get_center(),
            color=YELLOW
        )
        delta_label = Text("ΔPosition", font_size=20, color=YELLOW).next_to(delta_line, RIGHT, buff=0.5)
        
        self.play(
            Create(delta_line),
            Write(delta_label)
        )
        
        self.wait(10)

Manim Community v0.18.1

In [4]:
%%manim -qm ProtonNeutronSeparationWithPlot

from manim import *
import numpy as np

class ProtonNeutronSeparationWithPlot(Scene):
    def construct(self):
        # Scale factor for the left side animation
        scale_factor = 0.7
        left_shift = -3
        
         # Create grid on the right with switched x and y ranges
        grid = NumberPlane(
            x_range=[-2, 2, 0.5],    # Now horizontal (y)
            y_range=[-2, 4, 0.5],    # Now vertical (x)
            x_length=4,              # Horizontal extent
            y_length=6,              # Vertical extent
            background_line_style={
                "stroke_color": BLUE_D,
                "stroke_width": 1,
                "stroke_opacity": 0.2
            },
            axis_config={
                "stroke_opacity": 0.1,
                "include_numbers": False,
                "include_ticks": False
            }
        ).move_to([2, 0, 0])

        # Add labels with switched positions
        y_label = Text("Y (actual - predicted)", font_size=20).next_to(grid, DOWN)  # Now on bottom
        x_label = Text("X (actual - predicted)", font_size=20).rotate(90 * DEGREES).next_to(grid, LEFT)  # Now on left
        
        plot_title = Text("HCal Position Differences", font_size=24).next_to(grid, UP)
        
        # Add neutron and proton points (switched coordinates)
        neutron_dot = Dot(grid.c2p(0, 0), color=BLUE)  # At origin (no difference)
        proton_dot = Dot(grid.c2p(0, 2), color=RED)  # Only x-deflection (now vertical)
        
        # Add plot elements
        self.play(
            Create(grid),
            Write(x_label),
            Write(y_label),
            Write(plot_title)
        )
        
        # Add points with labels
        neutron_plot_label = Text("Neutron", font_size=24, color=BLUE).next_to(neutron_dot, RIGHT)
        proton_plot_label = Text("Proton", font_size=24, color=RED).next_to(proton_dot, RIGHT)
        
        self.play(
            Create(neutron_dot),
            Write(neutron_plot_label),
        )
        self.play(
            Create(proton_dot),
            Write(proton_plot_label),
        )
        
        self.wait(2)

Manim Community v0.18.1

In [5]:
%%manim -qm ProtonNeutronSeparationWithPlot
from manim import *
import numpy as np

class ProtonNeutronSeparationWithPlot(Scene):
    def construct(self):
        # Scale factor for the left side animation
        scale_factor = 0.7
        left_shift = -3
        
        # Create grid on the right with switched x and y ranges
        grid = NumberPlane(
            x_range=[-2, 2, 0.5],    # Now horizontal (y)
            y_range=[-2, 4, 0.5],    # Now vertical (x)
            x_length=4,              # Horizontal extent
            y_length=6,              # Vertical extent
            background_line_style={
                "stroke_color": BLUE_D,
                "stroke_width": 1,
                "stroke_opacity": 0.2
            },
            axis_config={
                "stroke_opacity": 0.1,
                "include_numbers": False,
                "include_ticks": False
            }
        ).move_to([2, 0, 0])

        # Add labels with switched positions
        y_label = Text("Y (actual - predicted)", font_size=20).next_to(grid, DOWN)
        x_label = Text("X (actual - predicted)", font_size=20).rotate(90 * DEGREES).next_to(grid, LEFT)
        plot_title = Text("HCal Position Differences", font_size=24).next_to(grid, UP)
        
        # Add initial points
        neutron_dot = Dot(grid.c2p(0, 0), color=BLUE)
        proton_dot = Dot(grid.c2p(0, 2), color=RED)
        
        # Add plot elements
        self.play(
            Create(grid),
            Write(x_label),
            Write(y_label),
            Write(plot_title),
            run_time=0.2
        )
        
        # Add initial points with labels
        neutron_plot_label = Text("Neutron", font_size=24, color=BLUE).next_to(neutron_dot, RIGHT)
        proton_plot_label = Text("Proton", font_size=24, color=RED).next_to(proton_dot, RIGHT)
        
        self.play(
            Create(neutron_dot),
            Write(neutron_plot_label),
        )
        self.play(
            Create(proton_dot),
            Write(proton_plot_label),
        )

        self.wait(2)


        self.play(
            FadeOut(neutron_dot),
            FadeOut(neutron_plot_label),
            FadeOut(proton_dot),
            FadeOut(proton_plot_label)
        )
        
        # Generate and add Gaussian distributed points
        n_points_p = 5
        n_points_n = 2  # number of points per batch
        n_batches = 100   # number of batches to add
        
        # Parameters for the distributions
        neutron_mean = [0, 0]
        proton_mean = [0, 2]
        neutron_cov = [[0.08, 0], [0, 0.08]]  # tighter distribution for neutrons
        proton_cov = [[0.11, 0], [0, 0.11]]  # slightly wider distribution for protons
        
        for batch in range(n_batches):
            # Generate neutron points
            neutron_points = np.random.multivariate_normal(neutron_mean, neutron_cov, n_points_n)
            neutron_dots = VGroup(*[
                Dot(grid.c2p(x, y), color=BLUE, radius=0.03)
                for x, y in neutron_points
            ])
            
            # Generate proton points
            proton_points = np.random.multivariate_normal(proton_mean, proton_cov, n_points_p)
            proton_dots = VGroup(*[
                Dot(grid.c2p(x, y), color=RED, radius=0.03)
                for x, y in proton_points
            ])
            
            # Add points with a nice fade-in animation
            self.play(
                AnimationGroup(
                    *[FadeIn(dot, run_time=0.1) for dot in neutron_dots],
                    *[FadeIn(dot, run_time=0.1) for dot in proton_dots],
                    lag_ratio=0.02
                )
            )
        
        self.wait(2)

Manim Community v0.18.1